In [ ]:
import pandas as pd

In [ ]:
from kiteconnect import KiteConnect 
kite = KiteConnect(api_key="XXXXX")

In [ ]:
print(kite.login_url())

In [ ]:
data = kite.generate_session("XXXXX", api_secret="XXXXX")
kite.set_access_token(data["access_token"])

In [ ]:
# Gets historical data from Kite Connect
def get_historical_data():
    """
    Implementation of the moving average strategy.
    We go through the historical records that 
    we received form Kite Connect, calculate moving average,
    and place a BUY or SELL order.
    """
    return kite.historical_data(instrument_token, from_date, to_date, interval)

In [ ]:
# Instrument token of nifty
instrument_token = "256265" 

# Dates between which we need historical data
from_date = "2020-06-01"
to_date = "2020-08-30"

# Interval(minute, day, 3 minute, 5 minute...)
interval = "5minute"

In [ ]:
records = get_historical_data()
hist_df = pd.DataFrame(records)

In [ ]:
pd.set_option('display.max_rows', 1000)

In [ ]:
distance_from_strike = int(input("Please enter the distance value from strike"))
underlying_instrument = str(input("Please enter instrument symbol or token number of instrument"))
instrument_expiry_date = str(input("please enter instruement expiry date e.g: mdd(use 0-9,O,M,D) and if month end then mmm"))

In [ ]:
ltp = kite.ltp(['NSE:NIFTY 50'])
underlying_instrument_ltp = ltp['NSE:NIFTY 50']['last_price']

In [ ]:
rem = underlying_instrument_ltp % 50
if rem % 50 <= 25:
    underlying_instrument_value = underlying_instrument_ltp - rem
else:
    underlying_instrument_value = underlying_instrument_ltp - rem + 50

In [ ]:
underlying_instrument_ltp,underlying_instrument_value

In [ ]:
OTMCall = underlying_instrument_value + distance_from_strike
OTMPut = underlying_instrument_value - distance_from_strike

In [ ]:
OTMCall,OTMPut

In [ ]:
OTMCall_ins_symbol = underlying_instrument +'20' + instrument_expiry_date + str(int(OTMCall)) +'CE'
OTMPut_ins_symbol = underlying_instrument+'20' + instrument_expiry_date + str(int(OTMPut)) +'PE'

In [ ]:
kite.ltp(['NFO:' + OTMCall_ins_symbol,'NFO:' + OTMPut_ins_symbol])

In [ ]:
underlying_instrument_ltp

In [ ]:
interval = 100
reference = 9700
sig = []
ref = []
is_green_brick = 1

#TrendSelling_with_Renko
for underlying_instrument_ltp in hist_df['close']:

    step = underlying_instrument_ltp - reference

    if(abs(step) > interval and step > 0) and (green_brick == 1):
        reference = underlying_instrument_ltp - (underlying_instrument_ltp % interval)
        green_brick = 1
        sig.append("put")
        ref.append(reference)
    elif (abs(step) > 2 * interval and step < 0):
        reference = underlying_instrument_ltp - (underlying_instrument_ltp % interval) + 125
        green_brick = 0
        sig.append("call")
        ref.append(reference)
    else:
        sig.append("No decision")
        ref.append(reference)

In [ ]:
#TrendSelling_with_Renko
for underlying_instrument_ltp in hist_df['close']:

    step = underlying_instrument_ltp - reference

    if step >= 100 and is_green_brick == 1:
        sig.append("green-green")
        reference = underlying_instrument_ltp - (underlying_instrument_ltp % interval)
        ref.append(reference)
        is_green_brick = 1 
    elif step <= -200 and is_green_brick == 1:
        reference = underlying_instrument_ltp - (underlying_instrument_ltp % interval) + interval
        sig.append("green-red")
        ref.append(reference)
        is_green_brick = 0
    elif step <= -100 and is_green_brick == 0:
        sig.append("red-red")
        reference = underlying_instrument_ltp - (underlying_instrument_ltp % interval) + interval  
        ref.append(reference)
        is_green_brick = 0
    elif step >= 200 and is_green_brick == 0:
        sig.append("red-green")
        reference = underlying_instrument_ltp - (underlying_instrument_ltp % interval)
        ref.append(reference)
        is_green_brick = 1 
    else:
        sig.append("No decision")
        ref.append(reference)

In [ ]:
hist_df['signal'] = sig
hist_df['ref'] = ref

In [ ]:
hist_df.to_csv("logic.csv", index=False)